# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = data.groupby(['CustomerID', 'ProductName'])['Quantity'].sum().reset_index()
df

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
matrix = pd.pivot_table(data, index=["ProductName"], columns=["CustomerID"], values=["Quantity"], aggfunc= 'sum').fillna(0)
matrix

Quantity                                \
CustomerID                           33    200   264   356   412   464     
ProductName                                                                
Anchovy Paste - 56 G Tube              0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mini Egg Roll, Shrimp      0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mushroom Tart              0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Sausage Rolls              0.0   0.0   0.0   0.0   0.0   0.0   
Apricots - Dried                       1.0   0.0   0.0   0.0   1.0   0.0   
...                                    ...   ...   ...   ...   ...   ...   
Yeast Dry - Fermipan                   0.0   0.0   0.0   0.0   0.0   0.0   
Yoghurt Tubes                          0.0   0.0   0.0   0.0   0.0   0.0   
Yogurt - Blueberry, 175 Gr             0.0   1.0   0.0   0.0   0.0   0.0   
Yogurt - French Vanilla                1.0   0.0   0.0   1.0   0.0   0.0   
Zucchini - Yellow                      0.0   0.0   0.0   0.0   0.0   0.0   

                                                           ...              \
CustomerID                        477   639   649   669    ... 97697 97753   
ProductName                                                ...               
Anchovy Paste - 56 G Tube           0.0   1.0   0.0   0.0  ...   0.0  25.0   
Appetizer - Mini Egg Roll, Shrimp   0.0   0.0   0.0   0.0  ...  25.0  25.0   
Appetizer - Mushroom Tart           0.0   1.0   0.0   0.0  ...  25.0   0.0   
Appetizer - Sausage Rolls           0.0   0.0   0.0   0.0  ...   0.0   0.0   
Apricots - Dried                    0.0   0.0   0.0   0.0  ...   0.0  25.0   
...                                 ...   ...   ...   ...  ...   ...   ...   
Yeast Dry - Fermipan                0.0   0.0   0.0   0.0  ...   0.0   0.0   
Yoghurt Tubes                       0.0   0.0   2.0   0.0  ...   0.0   0.0   
Yogurt - Blueberry, 175 Gr          0.0   0.0   0.0   0.0  ...  25.0   0.0   
Yogurt - French Vanilla             2.0   0.0   0.0   1.0  ...   0.0   0.0   
Zucchini - Yellow                   0.0   0.0   0.0   0.0  ...   0.0   0.0   

                                                                             \
CustomerID                        97769 97793 97900 97928 98069 98159 98185   
ProductName                                                                   
Anchovy Paste - 56 G Tube           0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mini Egg Roll, Shrimp   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mushroom Tart           0.0   0.0   0.0   0.0   0.0   0.0  25.0   
Appetizer - Sausage Rolls           0.0   0.0  25.0  25.0  25.0   0.0  25.0   
Apricots - Dried                    0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...                                 ...   ...   ...   ...   ...   ...   ...   
Yeast Dry - Fermipan                0.0  25.0   0.0   0.0   0.0   0.0   0.0   
Yoghurt Tubes                       0.0   0.0   0.0   0.0   0.0  25.0   0.0   
Yogurt - Blueberry, 175 Gr          0.0  25.0   0.0   0.0   0.0   0.0   0.0   
Yogurt - French Vanilla            25.0   0.0   0.0   0.0   0.0   0.0   0.0   
Zucchini - Yellow                   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

                                         
CustomerID                        98200  
ProductName                              
Anchovy Paste - 56 G Tube           0.0  
Appetizer - Mini Egg Roll, Shrimp   0.0  
Appetizer - Mushroom Tart           0.0  
Appetizer - Sausage Rolls           0.0  
Apricots - Dried                    0.0  
...                                 ...  
Yeast Dry - Fermipan                0.0  
Yoghurt Tubes                       0.0  
Yogurt - Blueberry, 175 Gr          0.0  
Yogurt - French Vanilla            25.0  
Zucchini - Yellow                   0.0  

[452 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
def similarity_matrix(df):
    return pd.DataFrame(
        data = 1 / (1 + squareform(pdist(df.transpose()))),
        index = df.columns,
        columns = df.columns
    )
similarities = similarity_matrix(matrix)
similarities.columns = similarities.columns.droplevel()
similarities_transp = similarities.transpose()
similarities_transp.columns = similarities_transp.columns.droplevel()
similarities_transp.columns

Int64Index([   33,   200,   264,   356,   412,   464,   477,   639,   649,
              669,
            ...
            97697, 97753, 97769, 97793, 97900, 97928, 98069, 98159, 98185,
            98200],
           dtype='int64', name='CustomerID', length=1000)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
top_5 = similarities[33].sort_values(ascending = False).head(6)
list_top_6 = []
for n in top_5.index:
    list_top_6.append(n[1])
list_top_5 = list_top_6[1:6]
list_top_5

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
df1 = df[df['CustomerID'].isin(list_top_5)]
df1

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
matrix1 = df1.pivot_table(index=["ProductName"], columns=["CustomerID"], values=["Quantity"], aggfunc= 'sum').fillna(0)
matrix1.columns = matrix1.columns.droplevel()
matrix1['Total'] = matrix1.sum(axis = 1)
matrix1.sort_values('Total', ascending = False).head(5)

CustomerID,264,2503,3305,3317,3535,Total
ProductName,,,,,,
Butter - Unsalted,0.0,2.0,0.0,1.0,0.0,3.0
Wine - Ej Gallo Sierra Valley,0.0,0.0,2.0,1.0,0.0,3.0
Towels - Paper / Kraft,1.0,0.0,1.0,0.0,1.0,3.0
Soup - Campbells Bean Medley,1.0,0.0,0.0,1.0,1.0,3.0
Wine - Blue Nun Qualitatswein,1.0,0.0,1.0,0.0,1.0,3.0


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [20]:
no_products = list(matrix[matrix[33] == 0].index)
df2 = matrix1[matrix1.index.isin(no_products)].sort_values('Total', ascending = False).head(5)
#df2.columns = df2.columns.droplevel()
recommend_products = list(df2.index)

#matrix.columns = matrix.columns.droplevel()
#matrix



## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [21]:
all_cost = {}
customers = list(set(data['CustomerID']))

list_top_6 = []
def f_list_top_5(top_5):
    for n in top_5.index:
        list_top_6.append(n[1])
    list_top_5 = list_top_6[1:6]
    return list_top_5

def product_recomendation(df, customer):
        top_5 = similarities[customer].sort_values(ascending = False).head(6)
        list_top_5 = f_list_top_5(top_5)
        df1 = df[df['CustomerID'].isin(list_top_5)]
        matrix1 = df1.pivot_table(index=["ProductName"], columns=["CustomerID"], values=["Quantity"], aggfunc= 'sum').fillna(0)
        matrix1.columns = matrix1.columns.droplevel()
        matrix1['Total'] = matrix1.sum(axis = 1)
        matrix1.sort_values('Total', ascending = False).head(5)
        no_products = list(matrix[matrix[customer] == 0].index)
        df2 = matrix1[matrix1.index.isin(no_products)].sort_values('Total', ascending = False).head(5)
        recommend_products = list(df2.index)
        return recommend_products

all_cost = {customer : product_recomendation(df, customer) for customer in customers}

df_all_cost = pd.DataFrame(all_cost)
df_all_cost.head()
    

,83973,59399,92168,49159,18441,22536,86028,75791,96272,32785,...,55281,75762,45042,59379,77810,77814,32753,20476,67582,65535
0,Cocoa Butter,Baking Powder,Baking Powder,Baking Powder,Baking Powder,Baking Powder,Baking Powder,Baking Powder,Baking Powder,Baking Powder,...,Baking Powder,Cocoa Butter,Baking Powder,Guinea Fowl,Baking Powder,Muffin Chocolate Individual Wrap,Baking Powder,Baking Powder,Cocoa Butter,Baking Powder
1,Veal - Slab Bacon,Guinea Fowl,Thermometer Digital,Thermometer Digital,Cocoa Butter,Guinea Fowl,Thermometer Digital,Cocoa Butter,Thermometer Digital,Muffin Chocolate Individual Wrap,...,Guinea Fowl,Muffin Chocolate Individual Wrap,Thermometer Digital,Thermometer Digital,Thermometer Digital,Guinea Fowl,Cocoa Butter,Thermometer Digital,Guinea Fowl,Thermometer Digital
2,Bandage - Fexible 1x3,Muffin Chocolate Individual Wrap,Muffin Chocolate Individual Wrap,Cocoa Butter,Guinea Fowl,Thermometer Digital,Guinea Fowl,Guinea Fowl,Guinea Fowl,Cocoa Butter,...,Thermometer Digital,Thermometer Digital,Cocoa Butter,Cocoa Butter,Guinea Fowl,Thermometer Digital,Guinea Fowl,Cocoa Butter,Thermometer Digital,Cocoa Butter
3,Ezy Change Mophandle,Cocoa Butter,Cocoa Butter,Veal - Slab Bacon,Veal - Slab Bacon,Muffin Chocolate Individual Wrap,Cocoa Butter,Thermometer Digital,Muffin Chocolate Individual Wrap,Guinea Fowl,...,Muffin Chocolate Individual Wrap,Veal - Slab Bacon,Muffin Chocolate Individual Wrap,Muffin Chocolate Individual Wrap,Cocoa Butter,Cocoa Butter,Thermometer Digital,Guinea Fowl,Muffin Chocolate Individual Wrap,Muffin Chocolate Individual Wrap
4,Pop Shoppe Cream Soda,Thermometer Digital,Veal - Slab Bacon,Lettuce - Frisee,Lettuce - Frisee,Lettuce - Frisee,Muffin Chocolate Individual Wrap,Muffin Chocolate Individual Wrap,Veal - Slab Bacon,Thermometer Digital,...,Cocoa Butter,Lettuce - Frisee,Veal - Slab Bacon,Veal - Slab Bacon,Veal - Slab Bacon,Lettuce - Frisee,Veal - Slab Bacon,Muffin Chocolate Individual Wrap,Lettuce - Frisee,Veal - Slab Bacon


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [22]:
df_all_cost_transp = df_all_cost.transpose()
df_all_cost_transp 

,0,1,2,3,4
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Pop Shoppe Cream Soda
59399,Baking Powder,Guinea Fowl,Muffin Chocolate Individual Wrap,Cocoa Butter,Thermometer Digital
92168,Baking Powder,Thermometer Digital,Muffin Chocolate Individual Wrap,Cocoa Butter,Veal - Slab Bacon
49159,Baking Powder,Thermometer Digital,Cocoa Butter,Veal - Slab Bacon,Lettuce - Frisee
18441,Baking Powder,Cocoa Butter,Guinea Fowl,Veal - Slab Bacon,Lettuce - Frisee
...,...,...,...,...,...
77814,Muffin Chocolate Individual Wrap,Guinea Fowl,Thermometer Digital,Cocoa Butter,Lettuce - Frisee
32753,Baking Powder,Cocoa Butter,Guinea Fowl,Thermometer Digital,Veal - Slab Bacon
20476,Baking Powder,Thermometer Digital,Cocoa Butter,Guinea Fowl,Muffin Chocolate Individual Wrap
67582,Cocoa Butter,Guinea Fowl,Thermometer Digital,Muffin Chocolate Individual Wrap,Lettuce - Frisee


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [28]:
def similarity_matrix_metric(df, metric):
    return pd.DataFrame(
        data = 1 / (1 + squareform(pdist(df.transpose(), metric))),
        index = df.columns,
        columns = df.columns
    )
similarities_cosine = similarity_matrix_metric(matrix, 'cosine')
# similarities_cosine.columns = similarities_cosine.columns.droplevel()
# similarities_transp = similarities_cosine.transpose()
# similarities_transp#.columns = similarities_transp.columns.droplevel()
# # similarities_transp.columns
similarities_cosine

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.513809,0.521925,0.560225,0.530406,0.536686,0.542873,0.540381,0.533190,0.535224,0.526495,...,0.532674,0.539309,0.522883,0.542570,0.569539,1.000000,0.562050,0.543657,0.531363,0.519127
98069,0.528795,0.573177,0.548538,0.520138,0.519628,0.520835,0.534221,0.525789,0.531918,0.541402,...,0.529233,0.535979,0.530224,0.518742,0.554911,0.562050,1.000000,0.548011,0.542143,0.536728
98159,0.520208,0.537284,0.538340,0.536202,0.538710,0.533856,0.535601,0.535704,0.544926,0.535721,...,0.542777,0.556893,0.535487,0.521507,0.541404,0.543657,0.548011,1.000000,0.543258,0.531447


In [36]:
# similarities_cosine.columns

all_cost = {}
customers = list(set(data['CustomerID']))

list_top_6 = []
def f_list_top_5(top_5):
    for n in top_5.index:
        list_top_6.append(n[1])
    list_top_5 = list_top_6[1:6]
    return list_top_5

def product_recomendation(df, customer):
        top_5 = similarities_cosine[customer].sort_values(ascending = False).head(6)
        list_top_5 = f_list_top_5(top_5)
        df1 = df[df['CustomerID'].isin(list_top_5)]
        matrix1 = df1.pivot_table(index=["ProductName"], columns=["CustomerID"], values=["Quantity"], aggfunc= 'sum').fillna(0)
        matrix1.columns = matrix1.columns.droplevel()
        matrix1['Total'] = matrix1.sum(axis = 1)
        matrix1.sort_values('Total', ascending = False).head(5)
        no_products = list(matrix[matrix[customer] == 0].index)
        df2 = matrix1[matrix1.index.isin(no_products)].sort_values('Total', ascending = False).head(5)
        recommend_products = list(df2.index)
        return recommend_products

all_cost = {customer : product_recomendation(df, customer) for customer in customers}

df_all_cost = pd.DataFrame(all_cost)
df_all_cost.head()